In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
!unzip "/gdrive/My Drive/DBM Mask RCNN/Updated New Data.zip"

Archive:  /gdrive/My Drive/DBM Mask RCNN/Updated New Data.zip
   creating: Updated New Data/annots/
  inflating: Updated New Data/annots/1.xml  
  inflating: Updated New Data/annots/10.xml  
  inflating: Updated New Data/annots/100.xml  
  inflating: Updated New Data/annots/101.xml  
  inflating: Updated New Data/annots/102.xml  
  inflating: Updated New Data/annots/103.xml  
  inflating: Updated New Data/annots/104.xml  
  inflating: Updated New Data/annots/105.xml  
  inflating: Updated New Data/annots/106.xml  
  inflating: Updated New Data/annots/107.xml  
  inflating: Updated New Data/annots/108.xml  
  inflating: Updated New Data/annots/109.xml  
  inflating: Updated New Data/annots/11.xml  
  inflating: Updated New Data/annots/110.xml  
  inflating: Updated New Data/annots/111.xml  
  inflating: Updated New Data/annots/112.xml  
  inflating: Updated New Data/annots/113.xml  
  inflating: Updated New Data/annots/114.xml  
  inflating: Updated New Data/annots/115.xml  
  inflating

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageFile
import xml.etree
import xml.etree.ElementTree as ET
from pathlib import Path

In [ ]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
def extract_bboxes(xml_path):
  tree = xml.etree.ElementTree.parse(xml_path)
  root = tree.getroot()

  boxes = list()
  for box in root.findall('.//bndbox'):
    xmin = int(box.find('xmin').text)
    ymin = int(box.find('ymin').text)
    xmax = int(box.find('xmax').text)
    ymax = int(box.find('ymax').text)
    coors = [xmin, ymin, xmax, ymax]
    boxes.append(coors)

  width = int(root.find('.//size/width').text)
  height = int(root.find('.//size/height').text)
  return boxes, width, height

In [ ]:
def create_labelimg_xml(image_path, all_boxes):

  image_path = Path(image_path)
  img = np.array(Image.open(image_path).convert('RGB'))

  annotation = ET.Element('annotation')
  ET.SubElement(annotation, 'folder').text = str(image_path.parent.name)
  ET.SubElement(annotation, 'filename').text = str(image_path.name)
  ET.SubElement(annotation, 'path').text = str(image_path)

  source = ET.SubElement(annotation, 'source')
  ET.SubElement(source, 'database').text = 'Unknown'

  size = ET.SubElement(annotation, 'size')
  ET.SubElement(size, 'width').text = str(img.shape[1])
  ET.SubElement(size, 'height').text = str(img.shape[0])
  ET.SubElement(size, 'depth').text = str(img.shape[2])

  ET.SubElement(annotation, 'segmented').text = '0'

  if all_boxes:
    for box in all_boxes:
      ymin, xmin, ymax, xmax = box[1], box[0], box[3], box[2]

      object = ET.SubElement(annotation, 'object')
      ET.SubElement(object, 'name').text = 'DBM'
      ET.SubElement(object, 'pose').text = 'Unspecified'
      ET.SubElement(object, 'truncated').text = '0'
      ET.SubElement(object, 'difficult').text = '0'

      bndbox = ET.SubElement(object, 'bndbox')
      ET.SubElement(bndbox, 'xmin').text = str(xmin)
      ET.SubElement(bndbox, 'ymin').text = str(ymin)
      ET.SubElement(bndbox, 'xmax').text = str(xmax)
      ET.SubElement(bndbox, 'ymax').text = str(ymax)

  tree = ET.ElementTree(annotation)
  return tree

In [ ]:
def find_nearest_cropline(width, height, x_occupied, y_occupied):

  selected_width = int(width/2)

  if selected_width not in x_occupied:
    cut_width = selected_width
    cut_width_found = True

  else:
    cut_width_found = False
    selected_width_left = selected_width
    selected_width_right = selected_width

  while not cut_width_found:
    selected_width_left = selected_width_left - 1
    selected_width_right = selected_width_right + 1

    if selected_width_left not in x_occupied:
      cut_width = selected_width_left
      cut_width_found = True

    if selected_width_right not in x_occupied:
      cut_width = selected_width_right
      cut_width_found = True


  selected_height = int(height/2)

  if selected_height not in y_occupied:
    cut_height = selected_height
    cut_width_found = True

  else:
    cut_width_found = False
    selected_height_down = selected_height
    selected_height_up = selected_height

  while not cut_width_found:
    selected_height_down = selected_height_down + 1
    selected_height_up = selected_height_up - 1

    if selected_height_down not in y_occupied:
      cut_height = selected_height_down
      cut_width_found = True

    if selected_height_up not in y_occupied:
      cut_height = selected_height_up
      cut_width_found = True

  return cut_width, cut_height

In [ ]:
from PIL import Image, ImageOps

In [ ]:
dataset_dir = "Updated New Data"

images_dir = dataset_dir + '/images/'
annotations_dir = dataset_dir + '/annots/'

for filename in range(1, 739):
  x_occupied = []
  y_occupied = []

  print(filename)
  image_path = images_dir + str(filename) + ".jpg"
  if not os.path.isfile(image_path):
    image_path = images_dir + str(filename) + ".jpeg"

  xml_path = annotations_dir + str(filename) + ".xml"
  #img = plt.imread(image_path)
  imag = Image.open(image_path)

  exif = imag.getexif()
  # Remove all exif tags
  for k in exif.keys():
      if k != 0x0112:
          exif[k] = None # If I don't set it to None first (or print it) the del fails for some reason. 
          del exif[k]
  # Put the new exif object in the original image
  new_exif = exif.tobytes()
  imag.info["exif"] = new_exif
  # Rotate the image
  img2 = ImageOps.exif_transpose(imag)


  #img2 = ImageOps.exif_transpose(imag)
  img = np.array(img2)

  boxes, width, height = extract_bboxes(xml_path)
  #print(width, height)

  for box in boxes:
    for num in range(box[0], box[2] + 1):
      x_occupied.append(num)
    for num in range(box[1], box[3] + 1):
      y_occupied.append(num)

  cut_width, cut_height = find_nearest_cropline(width, height, x_occupied, y_occupied)
  #print(cut_width, cut_height)

  cropped_image_topleft = img[0:cut_height, 0:cut_width]
  cropped_image_topright = img[0:cut_height, cut_width:width]
  cropped_image_bottomleft = img[cut_height:height, 0:cut_width]
  cropped_image_bottomright = img[cut_height:height, cut_width:width]


  plt.imsave('//gdrive//My Drive//DBM Mask RCNN//split in 4//%s.png' % str((4 * int(filename)) - 3), cropped_image_topleft)
  plt.imsave('//gdrive//My Drive//DBM Mask RCNN//split in 4//%s.png' % str((4 * int(filename)) - 2), cropped_image_topright)
  plt.imsave('//gdrive//My Drive//DBM Mask RCNN//split in 4//%s.png' % str((4 * int(filename)) - 1), cropped_image_bottomleft)
  plt.imsave('//gdrive//My Drive//DBM Mask RCNN//split in 4//%s.png' % str((4 * int(filename))), cropped_image_bottomright)
  

  top_left_boxes = []
  bottom_left_boxes = []
  top_right_boxes = []
  bottom_right_boxes = []

  for box in boxes:
    x = []
    y = []
    left = []
    right = []
    top = []
    bottom = []
    for num in range(box[0], box[2] + 1):
      x.append(num)
    for num in range(box[1], box[3] + 1):
      y.append(num)

    for num in range(0, cut_width):
      left.append(num)
    for num in range(cut_width, width):
      right.append(num)
    for num in range(0, cut_height):
      top.append(num)
    for num in range(cut_height, height):
      bottom.append(num)

    set_x = set(x)
    set_y = set(y)
    set_left = set(left)
    set_right = set(right)
    set_top = set(top)
    set_bottom = set(bottom)

    if set_x.intersection(set_left):
      if set_y.intersection(set_top):
        #print("top left")
        #print("height: ", cropped_image_topleft.shape[0])
        #print("width: ", cropped_image_topleft.shape[1])
        #print("Old & New Box: ", box)

        if box[0] > cropped_image_topleft.shape[1]:
          print("1st----------------------------------------------------------------")
        if box[1] > cropped_image_topleft.shape[0]:
          print("2nd----------------------------------------------------------------")
        if box[2] > cropped_image_topleft.shape[1]:
          print("3rd----------------------------------------------------------------")
        if box[3] > cropped_image_topleft.shape[0]:
          print("4th----------------------------------------------------------------")

        top_left_boxes.append(box)

    if set_x.intersection(set_left):
      if set_y.intersection(set_bottom):
        #print("bottom left")
        #print("height: ", cropped_image_bottomleft.shape[0])
        #print("width: ", cropped_image_bottomleft.shape[1])
        #print("Old Box: ", box)
        box[1] = box[1] - cut_height
        box[3] = box[3] - cut_height
        #print("New Box: ", box)

        if box[0] > cropped_image_bottomleft.shape[1]:
          print("1st----------------------------------------------------------------")
        if box[1] > cropped_image_bottomleft.shape[0]:
          print("2nd----------------------------------------------------------------")
        if box[2] > cropped_image_bottomleft.shape[1]:
          print("3rd----------------------------------------------------------------")
        if box[3] > cropped_image_bottomleft.shape[0]:
          print("4th----------------------------------------------------------------")

        bottom_left_boxes.append(box)

    if set_x.intersection(set_right):
      if set_y.intersection(set_top):
        #print("top right")
        #print("height: ", cropped_image_topright.shape[0])
        #print("width: ", cropped_image_topright.shape[1])
        #print("Old Box: ", box)
        box[0] = box[0] - cut_width
        box[2] = box[2] - cut_width
        #print("New Box: ", box)

        if box[0] > cropped_image_topright.shape[1]:
          print("1st----------------------------------------------------------------")
        if box[1] > cropped_image_topright.shape[0]:
          print("2nd----------------------------------------------------------------")
        if box[2] > cropped_image_topright.shape[1]:
          print("3rd----------------------------------------------------------------")
        if box[3] > cropped_image_topright.shape[0]:
          print("4th----------------------------------------------------------------")
        
        top_right_boxes.append(box)

    if set_x.intersection(set_right):
      if set_y.intersection(set_bottom):
        #print("bottom right")
        #print("height: ", cropped_image_bottomright.shape[0])
        #print("width: ", cropped_image_bottomright.shape[1])
        #print("Old Box: ", box)
        box[0] = box[0] - cut_width
        box[1] = box[1] - cut_height
        box[2] = box[2] - cut_width
        box[3] = box[3] - cut_height
        #print("New Box: ", box)

        if box[0] > cropped_image_bottomright.shape[1]:
          print("1st----------------------------------------------------------------")
        if box[1] > cropped_image_bottomright.shape[0]:
          print("2nd----------------------------------------------------------------")
        if box[2] > cropped_image_bottomright.shape[1]:
          print("3rd----------------------------------------------------------------")
        if box[3] > cropped_image_bottomright.shape[0]:
          print("4th----------------------------------------------------------------")
        
        bottom_right_boxes.append(box)


    if box[0] < 0 or box[1] < 0 or box[2] < 0 or box[3] < 0:
      print("--------------------------------------coordinates less than 0----------------------------------------------------")


  tree = create_labelimg_xml('//gdrive//My Drive//DBM Mask RCNN//split in 4//%s.png' % str((4 * int(filename)) - 3), top_left_boxes)
  tree.write('/gdrive/My Drive/DBM Mask RCNN/split in 4/%s.xml' % str((4 * int(filename)) - 3))

  tree = create_labelimg_xml('//gdrive//My Drive//DBM Mask RCNN//split in 4//%s.png' % str((4 * int(filename)) - 2), top_right_boxes)
  tree.write('/gdrive/My Drive/DBM Mask RCNN/split in 4/%s.xml' % str((4 * int(filename)) - 2))

  tree = create_labelimg_xml('//gdrive//My Drive//DBM Mask RCNN//split in 4//%s.png' % str((4 * int(filename)) - 1), bottom_left_boxes)
  tree.write('/gdrive/My Drive/DBM Mask RCNN/split in 4/%s.xml' % str((4 * int(filename)) - 1))

  tree = create_labelimg_xml('//gdrive//My Drive//DBM Mask RCNN//split in 4//%s.png' % str(4 * int(filename)), bottom_right_boxes)
  tree.write('/gdrive/My Drive/DBM Mask RCNN/split in 4/%s.xml' % str((4 * int(filename))))

  '''
  print(top_left_boxes)
  print(bottom_left_boxes)
  print(top_right_boxes)
  print(bottom_right_boxes)

  plt.imshow(img)
  plt.show()
  plt.imshow(cropped_image_topleft)
  plt.show()
  plt.imshow(cropped_image_topright)
  plt.show()
  plt.imshow(cropped_image_bottomleft)
  plt.show()
  plt.imshow(cropped_image_bottomright)
  plt.show()
  '''

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
